In [1]:
filePath = 'audio/seven.wav'

- separate

In [ ]:
from spleeter.separator import Separator

def separate(path: str):
    print('start separate')

    separateMode = 5
    # setting spleeter separator
    separator = Separator(f'spleeter:{separateMode}stems')

    # start separate
    separator.separate_to_file(path, f'result_{separateMode}stems')

    print('end separate')

In [ ]:
separate(filePath)